In [1]:
import tensorflow as tf
import sqlite3 as sqlite

### 数据库格式


| time          | identity                 | platform | route    | method | status | duration |
| ------------- | ------------------------ | -------- | -------- | ------ | ------ | -------- |
| 1519303089604 | Aoj7QgXnFBRCkri/r9TjWA== | web      | /api/jwc | get    | 200    | 384      |


数据表创建如下：

```
CREATE TABLE stat ( time int not null, identity text not null, platform text not null, route text not null, method text not null, status int not null, duration int not null );
```

主要关注字段包括：

* **time** - 访问时间戳，Unix标准时间戳
* **identity** - 用户唯一指定身份标识，在小猴偷米的应用语境下和每一个学生一一对应
* **platform** - 用户访问平台，此处关注 <web|ws2-mina-ios|ws2-mina-android|ws2-android|ws2-ios> 五个平台
* **route** - 用户访问的业务对象，根据实际访问服务分类，此处关注小猴偷米核心业务中的相关模块
* **method** - HTTP请求方法 get | post

In [4]:
class Database:
    def __init__(self):
        self.db_conn = sqlite.connect('stat.db')
        self.db_cur = self.db_conn.cursor()
    
    # 用于获取所有用户的identity
    def get_user(self, n=None):
        if n is not None:
            result = self.db_cur.execute('SELECT DISTINCT identity FROM stat LIMIT %s' % n)
        else:
            result = self.db_cur.execute('SELECT DISTINCT identity FROM stat')
        user_list = []
        for i in result:
            user_list.append(i[0])
        return user_list
    
    # 查询指定identity用户的最近n条记录
    def get_log(self, identity, n):
        result = self.db_cur.execute('SELECT identity, time, platform, route, method FROM stat WHERE identity = "%s" ORDER BY time DESC LIMIT %d'%(identity, n))
        log_list = []
        for row in result:
            log_list.append({
                'identity': row[0],
                'time': row[1],
                'platform': row[2],
                'route': row[3],
                'method': row[4]
            })
        return log_list

In [3]:
# 数据库访问示例
db = Database()
user_list = db.get_user(10)
print(user_list)
log = db.get_log('O78YpLDK9YxIC9Lth8LauQ==', 100)
print(log)

['guest', 'Aoj7QgXnFBRCkri/r9TjWA==', 'nV8jrHHWRwHHI3oYtkT3Vw==', 'ykT33V+QjuUdi++Cl2wWnA==', 'O78YpLDK9YxIC9Lth8LauQ==', 'SDWeRbWwrdQwwU7zQiArvg==', 'FoHTwtdsRFjhVLGNuT8YFw==', 'JzVOXL/VkBfuBrJun2ssyA==', 'W9YKSShiyokq0aKEE+gCug==', '3PNgL1l4ujOcoML1PHg0hA==']
[{'identity': 'O78YpLDK9YxIC9Lth8LauQ==', 'time': 1526528030138, 'platform': 'web', 'route': '/api/curriculum', 'method': 'get'}, {'identity': 'O78YpLDK9YxIC9Lth8LauQ==', 'time': 1526528029729, 'platform': 'web', 'route': '/api/phylab', 'method': 'get'}, {'identity': 'O78YpLDK9YxIC9Lth8LauQ==', 'time': 1526528029729, 'platform': 'web', 'route': '/api/library', 'method': 'get'}, {'identity': 'O78YpLDK9YxIC9Lth8LauQ==', 'time': 1526528029729, 'platform': 'web', 'route': '/api/notice', 'method': 'get'}, {'identity': 'O78YpLDK9YxIC9Lth8LauQ==', 'time': 1526528029728, 'platform': 'web', 'route': '/api/term', 'method': 'get'}, {'identity': 'O78YpLDK9YxIC9Lth8LauQ==', 'time': 1526528029728, 'platform': 'web', 'route': '/api/exam', 'met